In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from scripts.utils import *
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from model.SE_DenseNet import SE_Dense_Model

#### Using weights

In [ ]:
# Init hyperparameters
epoch= 20
batch_size= 32
input_shape= (128, 128, 1)
using_class_weight= True
num_classes= 5

In [ ]:
# load data
train_data, val_data, test_data= get_data('data_1', input_shape, batch_size)
model= SE_Dense_Model(input_shape= input_shape, num_classes= num_classes)
model_path = "/media/data3/users/longnd/ML_prj/checkpoints/cnn.h5"
checkpoint = ModelCheckpoint(filepath= model_path,
                            save_weights_only= True,
                            monitor= 'val_f1_score',
                            save_best_only= True, verbose=1)

earlystop = EarlyStopping(monitor= 'val_f1_score', 
                          min_delta= 0.01, 
                          patience= 15,
                          verbose= 1,
                          restore_best_weights= True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_f1_score', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.00000001)

In [ ]:
# train model
class_weight= get_balanced_weight(train_data)
if using_class_weight:
    history = model.fit(
        train_data, 
        validation_data= val_data, 
        epochs=20, 
        class_weight=class_weight,
        callbacks=[earlystop, checkpoint, learning_rate_reduction])
else:
    history = model.fit(
        train_data, 
        validation_data= val_data, 
        epochs=20, 
        callbacks=[earlystop, checkpoint, learning_rate_reduction])

In [ ]:
# save history
save_history_to_csv(history, 'ourss.csv')

In [ ]:
# plot history
plot_history(history)

In [ ]:
# print result
get_report(model, test_data)

#### Using GAN